# Wildfire Smoke → PM2.5 Exposure Mapping + Short-Term Forecasting

## Project goal
Wildfire smoke can cause rapid increases in fine particulate matter (PM2.5), creating acute health risk.

This project builds an end-to-end, reproducible ML pipeline to:
1. **Estimate smoke-related PM2.5 exposure** (station-based + spatial mapping)
2. **Forecast short-term PM2.5** (next-day / next-24h), using time-safe features
3. **Quantify operational trade-offs** (Precision/Recall, threshold tuning, calibration)
4. **Deliver deployable artifacts**: clean dataset, trained model, evaluation plots, and an exposure map product

## Key deliverables
- EDA + quality checks + missingness analysis
- Feature engineering (lags, trends, meteorology, smoke flags, seasonality)
- Train/validation/test with time-aware splits (no leakage)
- Baselines + stronger models (linear + tree-based)
- Probability calibration & threshold tuning for rare events (exceedance detection)
- Exposure mapping: station values → gridded surface (simple interpolation for portfolio)
- Clear limitations and next steps

> NOTE: This notebook is designed to run in two modes:
- **Mode A (recommended)**: you provide station PM2.5 CSV(s) and optionally smoke polygons.
- **Mode B (optional)**: you enable downloads/API pulls (Open-Meteo for weather; NOAA HMS smoke polygons).

In [1]:
import os
import math
import warnings
from dataclasses import dataclass
from pathlib import Path
from typing import Optional, Tuple, List, Dict

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import (
    mean_absolute_error, mean_squared_error, r2_score,
    precision_recall_curve, average_precision_score,
    precision_score, recall_score, f1_score, confusion_matrix,
    brier_score_loss
)
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.calibration import CalibratedClassifierCV, calibration_curve

warnings.filterwarnings("ignore")
plt.rcParams["figure.dpi"] = 140

## Data requirements (minimum viable)

You need **station-level PM2.5** time series, ideally hourly. Minimum columns:

- `datetime` (timestamp)
- `lat`, `lon` (station location)
- `pm25` (PM2.5 concentration, µg/m³)
- `station_id` (unique id)

Optional but recommended:
- `state`, `county`, `site_name`
- `aqi` or additional pollutants (PM10, O3)

### Smoke signal options (choose one)
**Option 1 (simple):** A daily smoke index column you provide per station (`smoke_flag` or `smoke_density`).
**Option 2 (stronger):** NOAA HMS smoke polygons → spatial join to stations (we include a template).
**Option 3 (portfolio-grade):** satellite AOD fields (optional later).

### Meteorology (recommended)
Meteorology improves signal separation between “local pollution” and “smoke-driven spikes”.
We support:
- **Open-Meteo API** (no key) for temperature, wind, humidity, precipitation, PBL height proxies (limited)
- OR you load your own met CSV.

In this notebook we treat meteorology as optional: models still run without it.

In [ ]:
@dataclass
class Config:
    # Paths
    DATA_DIR: Path = Path("data")
    OUT_DIR: Path = Path("outputs")

    # Input files (set these)
    PM25_FILE: Path = Path("data/pm25_stations.csv")  # REQUIRED
    SMOKE_FILE: Optional[Path] = None                # optional: merged station-day smoke flags
    MET_FILE: Optional[Path] = None                  # optional: merged station met data

    # Forecast target setup
    # Use daily aggregation for stability (you can change to hourly later).
    AGG: str = "D"  # 'D' daily, 'H' hourly
    FORECAST_HORIZON_STEPS: int = 1  # 1 day ahead if AGG='D'

    # Exceedance threshold (for classification task)
    EXCEED_PM25: float = 35.0  # µg/m³ (EPA 24-hr standard)

    # Split dates (time-aware)
    TRAIN_END: str = "2021-12-31"
    VAL_END: str = "2022-12-31"   # test = after this
    MIN_HISTORY_STEPS: int = 14   # require at least 2 weeks history per station for features

cfg = Config()
cfg.OUT_DIR.mkdir(parents=True, exist_ok=True)
cfg.DATA_DIR.mkdir(parents=True, exist_ok=True)

cfg

## Step 1 — Load & validate PM2.5 station data

We first load station PM2.5 data and enforce:
- consistent timestamps
- numeric PM2.5
- unique station identifiers
- basic sanity bounds

This is a “quality gate”: if you get bad data here, everything downstream becomes unreliable.

In [ ]:
def load_pm25(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path)
    # Standardize column names
    df.columns = [c.strip().lower() for c in df.columns]

    required = {"datetime", "station_id", "lat", "lon", "pm25"}
    missing = required - set(df.columns)
    if missing:
        raise ValueError(f"PM2.5 file is missing columns: {missing}. Found: {df.columns.tolist()}")

    df["datetime"] = pd.to_datetime(df["datetime"], utc=True, errors="coerce")
    df = df.dropna(subset=["datetime", "station_id", "lat", "lon", "pm25"]).copy()
    df["pm25"] = pd.to_numeric(df["pm25"], errors="coerce")
    df = df.dropna(subset=["pm25"]).copy()

    # Basic bounds (keep wide; smoke can be high)
    df = df[(df["pm25"] >= 0) & (df["pm25"] <= 2000)].copy()

    # Ensure types
    df["station_id"] = df["station_id"].astype(str)
    df["lat"] = df["lat"].astype(float)
    df["lon"] = df["lon"].astype(float)

    return df

pm = load_pm25(cfg.PM25_FILE)
pm.head(), pm.shape

## Step 2 — Exploratory Data Analysis (EDA)

We answer:
- How many stations do we have?
- How long is the time series?
- Are there missing periods or sparse stations?
- Are there obvious seasonal patterns?
- Are extreme values plausible?

EDA is not “pretty pictures” — it’s how we prevent silent failure.

We will:
1. Plot station coverage (count over time)
2. Show distribution of PM2.5 (log + linear)
3. Plot example station time series
4. Summarize missingness after resampling

In [ ]:
# Basic summary
print("n_rows:", len(pm))
print("n_stations:", pm["station_id"].nunique())
print("time range:", pm["datetime"].min(), "→", pm["datetime"].max())

# Station count over time (raw timestamps)
pm_temp = pm.copy()
pm_temp["date"] = pm_temp["datetime"].dt.floor("D")
station_count = pm_temp.groupby("date")["station_id"].nunique()

plt.figure(figsize=(8,4))
plt.plot(station_count.index, station_count.values)
plt.title("Active stations over time")
plt.xlabel("Date")
plt.ylabel("# Stations with data")
plt.grid(True)
plt.show()

# PM2.5 distribution
vals = pm["pm25"].values

plt.figure(figsize=(7,4))
plt.hist(vals, bins=80)
plt.title("PM2.5 distribution (linear)")
plt.xlabel("PM2.5 (µg/m³)")
plt.ylabel("Count")
plt.grid(True)
plt.show()

plt.figure(figsize=(7,4))
plt.hist(np.log1p(vals), bins=80)
plt.title("PM2.5 distribution (log1p)")
plt.xlabel("log(1 + PM2.5)")
plt.ylabel("Count")
plt.grid(True)
plt.show()

# Example stations
example_ids = pm["station_id"].drop_duplicates().head(3).tolist()
for sid in example_ids:
    d = pm[pm["station_id"] == sid].sort_values("datetime").copy()
    # downsample for plot speed
    d = d.set_index("datetime")["pm25"].resample("D").mean().reset_index()
    plt.figure(figsize=(9,3))
    plt.plot(d["datetime"], d["pm25"])
    plt.title(f"Example station {sid} — daily mean PM2.5")
    plt.xlabel("Date")
    plt.ylabel("PM2.5 (µg/m³)")
    plt.grid(True)
    plt.show()

## Step 3 — Daily aggregation + forecasting target

Why daily?
- Smoke health impact is often framed in 24-hour exposure.
- Many stations have missing hourly points; daily aggregation stabilizes.
- It makes forecasting and evaluation cleaner.

We compute per station per day:
- `pm25_mean` (daily mean)
- `pm25_max` (daily max)
- `exceed` = 1 if `pm25_mean >= 35` (24-hr standard default; adjustable)

Forecast target:
- `y_reg` = PM2.5 mean at **t + 1 day**
- `y_cls` = exceedance at **t + 1 day**

IMPORTANT: Features must only use information available at time t (time-safe).

In [ ]:
def aggregate_daily(pm: pd.DataFrame, agg: str = "D") -> pd.DataFrame:
    df = pm.copy()
    df = df.sort_values(["station_id", "datetime"])
    df = df.set_index("datetime")

    # Station metadata (static)
    meta = df.groupby("station_id")[["lat", "lon"]].first()

    daily = (
        df.groupby("station_id")["pm25"]
          .resample(agg)
          .agg(pm25_mean="mean", pm25_max="max", pm25_count="count")
          .reset_index()
    )
    daily = daily.merge(meta.reset_index(), on="station_id", how="left")
    daily = daily.rename(columns={"datetime": "date"})
    return daily

daily = aggregate_daily(pm, cfg.AGG)
daily.head(), daily.shape

In [ ]:
def add_targets(daily: pd.DataFrame, horizon_steps: int, exceed_thr: float) -> pd.DataFrame:
    df = daily.sort_values(["station_id", "date"]).copy()
    df["exceed_today"] = (df["pm25_mean"] >= exceed_thr).astype(int)

    # Targets at t+h
    df["y_reg"] = df.groupby("station_id")["pm25_mean"].shift(-horizon_steps)
    df["y_cls"] = df.groupby("station_id")["exceed_today"].shift(-horizon_steps)

    # Drop rows where target is unknown (tail)
    df = df.dropna(subset=["y_reg", "y_cls"]).copy()
    df["y_cls"] = df["y_cls"].astype(int)
    return df

df0 = add_targets(daily, cfg.FORECAST_HORIZON_STEPS, cfg.EXCEED_PM25)
df0.head(), df0.shape

## Step 4 — Feature engineering (time-safe)

We build predictors that are available at time t:

### A) History / momentum features
- Lags: yesterday / 2 days / 4 days PM2.5
- Rolling mean: last 4 days average
- Deltas: change over last 1 day, 2 days, 4 days

These capture persistence and sudden jumps (common with smoke intrusions).

### B) Spatiotemporal context
- latitude, longitude
- day-of-year (seasonality)
- weekday / month (optional)

### C) Smoke signal (optional)
- station-day `smoke_flag` (0/1) or `smoke_density`
If provided, this is often the strongest feature.

### D) Meteorology (optional)
- wind speed / direction proxy
- temperature
- humidity
- precipitation
This helps separate smoke transport from local sources.

We keep everything “leakage-safe”: no peeking into future days.

In [ ]:
def make_features(df: pd.DataFrame) -> pd.DataFrame:
    d = df.sort_values(["station_id", "date"]).copy()

    # Calendar
    d["date_local"] = pd.to_datetime(d["date"]).dt.tz_convert(None) if str(d["date"].dtype).startswith("datetime64") else pd.to_datetime(d["date"])
    d["doy"] = d["date_local"].dt.dayofyear
    d["month"] = d["date_local"].dt.month
    d["dow"] = d["date_local"].dt.dayofweek

    # Lags of pm25_mean
    for k in [1, 2, 4, 7]:
        d[f"pm25_lag_{k}"] = d.groupby("station_id")["pm25_mean"].shift(k)

    # Rolling mean (past only)
    d["pm25_rollmean_4"] = (
        d.groupby("station_id")["pm25_mean"]
         .shift(1)
         .rolling(window=4, min_periods=2)
         .mean()
         .reset_index(level=0, drop=True)
    )

    # Deltas (momentum)
    d["dpm25_1"] = d["pm25_lag_1"] - d["pm25_lag_2"]
    d["dpm25_2"] = d["pm25_lag_2"] - d["pm25_lag_4"]
    d["dpm25_4"] = d["pm25_lag_1"] - d["pm25_lag_7"]

    return d

df1 = make_features(df0)

# Require history
feature_cols_preview = [c for c in df1.columns if c.startswith("pm25_") or c.startswith("dpm25_")] + ["lat","lon","doy","month","dow"]
df1[feature_cols_preview].isna().mean().sort_values(ascending=False).head(15)

## Step 5 (optional) — Merge smoke + meteorology

If you have these, merge them now (station_id + date):

- Smoke: `smoke_flag` or `smoke_density`
- Meteorology: `wind_speed`, `temperature`, `humidity`, `precip`, etc.

If you *do not* have them yet, skip this section — the notebook still works.

### Recommended practical approach (portfolio-friendly)
- Start with PM2.5-only model (lags + seasonality).
- Add smoke_flag later → show clear performance improvement.
- Add meteorology after → demonstrate “scientific features” thinking.

In [ ]:
def merge_optional(df: pd.DataFrame, smoke_file: Optional[Path], met_file: Optional[Path]) -> pd.DataFrame:
    out = df.copy()

    if smoke_file is not None and Path(smoke_file).exists():
        smoke = pd.read_csv(smoke_file)
        smoke.columns = [c.strip().lower() for c in smoke.columns]
        if "date" in smoke.columns:
            smoke["date"] = pd.to_datetime(smoke["date"], utc=True, errors="coerce")
        if not {"station_id","date"}.issubset(smoke.columns):
            raise ValueError("SMOKE_FILE must contain station_id and date columns.")
        smoke["station_id"] = smoke["station_id"].astype(str)
        out = out.merge(smoke, on=["station_id","date"], how="left")
        print("Merged smoke features:", [c for c in smoke.columns if c not in ["station_id","date"]])

    if met_file is not None and Path(met_file).exists():
        met = pd.read_csv(met_file)
        met.columns = [c.strip().lower() for c in met.columns]
        if "date" in met.columns:
            met["date"] = pd.to_datetime(met["date"], utc=True, errors="coerce")
        if not {"station_id","date"}.issubset(met.columns):
            raise ValueError("MET_FILE must contain station_id and date columns.")
        met["station_id"] = met["station_id"].astype(str)
        out = out.merge(met, on=["station_id","date"], how="left")
        print("Merged met features:", [c for c in met.columns if c not in ["station_id","date"]])

    return out

df2 = merge_optional(df1, cfg.SMOKE_FILE, cfg.MET_FILE)
df2.shape

## Step 6 — Train/Validation/Test split (time-aware)

We do **chronological splits** to simulate real forecasting:

- Train: up to `TRAIN_END`
- Validation: `TRAIN_END` → `VAL_END`
- Test: after `VAL_END`

Why this matters:
- Random splits leak seasonality and smoke episodes across splits.
- Time splits reflect real deployment: train on past, predict future.

In [ ]:
def time_split(df: pd.DataFrame, train_end: str, val_end: str) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    d = df.copy()
    d["date"] = pd.to_datetime(d["date"], utc=True)
    train_end = pd.to_datetime(train_end, utc=True)
    val_end = pd.to_datetime(val_end, utc=True)

    train = d[d["date"] <= train_end].copy()
    val = d[(d["date"] > train_end) & (d["date"] <= val_end)].copy()
    test = d[d["date"] > val_end].copy()

    return train, val, test

train_df, val_df, test_df = time_split(df2, cfg.TRAIN_END, cfg.VAL_END)

print(train_df.shape, val_df.shape, test_df.shape)
print("Train date range:", train_df["date"].min(), "→", train_df["date"].max())
print("Val date range:", val_df["date"].min(), "→", val_df["date"].max())
print("Test date range:", test_df["date"].min(), "→", test_df["date"].max())

In [ ]:
# Choose feature columns (dynamic based on optional merges)
base_features = [
    "lat", "lon", "doy", "month", "dow",
    "pm25_mean", "pm25_max",
    "pm25_lag_1","pm25_lag_2","pm25_lag_4","pm25_lag_7",
    "pm25_rollmean_4",
    "dpm25_1","dpm25_2","dpm25_4",
    "pm25_count"
]

optional_features = []
for c in df2.columns:
    if c in ["smoke_flag","smoke_density","smoke_cat"]:
        optional_features.append(c)
    if c in ["wind_speed","wind_gust","temperature","humidity","precip","pressure"]:
        optional_features.append(c)

cat_features = ["station_id"]  # station fixed effects (helps)
num_features = [c for c in base_features + optional_features if c in df2.columns]

target_reg = "y_reg"
target_cls = "y_cls"

print("Numeric features:", num_features)
print("Categorical features:", cat_features)

## Step 7 — Modeling strategy

We run two tasks:

### Task A: Regression (forecast PM2.5 level)
Models:
- **Ridge Regression** (strong baseline, interpretable)
- **RandomForestRegressor** (nonlinear, robust)
- **HistGradientBoostingRegressor** (strong tabular performance)

Metrics:
- MAE (primary)
- RMSE
- R²

### Task B: Classification (forecast exceedance)
We convert PM2.5 forecast into an operational decision:
- `y_cls = 1` if next-day PM2.5 mean ≥ 35 µg/m³.

Models:
- Logistic Regression (with scaling)
- Calibrated Logistic Regression (more reliable probabilities)

Metrics:
- PR-AUC (Average Precision) — best for rare events
- Precision / Recall / F1
- Confusion matrix
- Threshold tuning to choose an operating point

In [ ]:
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_features),
        ("cat", categorical_transformer, cat_features),
    ],
    remainder="drop"
)

In [ ]:
def eval_regression(y_true, y_pred) -> Dict[str, float]:
    mae = mean_absolute_error(y_true, y_pred)
    rmse = math.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    return {"MAE": mae, "RMSE": rmse, "R2": r2}

X_train = train_df[num_features + cat_features]
y_train = train_df[target_reg].astype(float)

X_val = val_df[num_features + cat_features]
y_val = val_df[target_reg].astype(float)

X_test = test_df[num_features + cat_features]
y_test = test_df[target_reg].astype(float)

reg_models = {
    "Ridge": Ridge(alpha=1.0, random_state=0),
    "RandomForest": RandomForestRegressor(n_estimators=300, random_state=0, n_jobs=-1, max_depth=None),
    "HistGB": HistGradientBoostingRegressor(random_state=0, max_depth=6)
}

reg_results = []
reg_fit = {}

for name, model in reg_models.items():
    pipe = Pipeline(steps=[("preprocess", preprocess), ("model", model)])
    pipe.fit(X_train, y_train)

    pred_val = pipe.predict(X_val)
    pred_test = pipe.predict(X_test)

    res = {"model": name}
    res.update({f"VAL_{k}": v for k, v in eval_regression(y_val, pred_val).items()})
    res.update({f"TEST_{k}": v for k, v in eval_regression(y_test, pred_test).items()})
    reg_results.append(res)
    reg_fit[name] = pipe

reg_table = pd.DataFrame(reg_results).sort_values("VAL_MAE")
reg_table

## Regression diagnostics

We visualize:
- predicted vs actual (test)
- error distribution
- a time slice for one station (forecast realism)

These plots catch systematic bias (e.g., underpredicting high-smoke spikes).

In [ ]:
best_reg_name = reg_table.iloc[0]["model"]
best_reg = reg_fit[best_reg_name]
print("Best regression model by VAL_MAE:", best_reg_name)

pred_test = best_reg.predict(X_test)

# Pred vs actual
plt.figure(figsize=(5,5))
plt.scatter(y_test, pred_test, s=8, alpha=0.4)
mx = max(y_test.max(), pred_test.max())
plt.plot([0, mx], [0, mx])
plt.title(f"Predicted vs Actual (TEST) — {best_reg_name}")
plt.xlabel("Actual PM2.5")
plt.ylabel("Predicted PM2.5")
plt.grid(True)
plt.show()

# Residuals
resid = pred_test - y_test
plt.figure(figsize=(7,4))
plt.hist(resid, bins=60)
plt.title("Residual distribution (TEST)")
plt.xlabel("Prediction error (pred - actual)")
plt.ylabel("Count")
plt.grid(True)
plt.show()

# Station slice
sid = test_df["station_id"].drop_duplicates().iloc[0]
slice_df = test_df[test_df["station_id"] == sid].sort_values("date").copy()
Xs = slice_df[num_features + cat_features]
slice_df["pred_pm25"] = best_reg.predict(Xs)

plt.figure(figsize=(10,3))
plt.plot(slice_df["date"], slice_df["y_reg"], label="Actual next-day PM2.5")
plt.plot(slice_df["date"], slice_df["pred_pm25"], label="Predicted next-day PM2.5")
plt.title(f"Station slice — {sid} (TEST)")
plt.xlabel("Date")
plt.ylabel("PM2.5")
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
X_train_c = train_df[num_features + cat_features]
y_train_c = train_df[target_cls].astype(int)

X_val_c = val_df[num_features + cat_features]
y_val_c = val_df[target_cls].astype(int)

X_test_c = test_df[num_features + cat_features]
y_test_c = test_df[target_cls].astype(int)

base_clf = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", LogisticRegression(max_iter=2000, class_weight="balanced", n_jobs=-1))
])

base_clf.fit(X_train_c, y_train_c)

# Calibrated version (better probability estimates)
cal_clf = CalibratedClassifierCV(
    estimator=Pipeline(steps=[
        ("preprocess", preprocess),
        ("model", LogisticRegression(max_iter=2000, class_weight="balanced", n_jobs=-1))
    ]),
    method="isotonic",  # use "sigmoid" if isotonic overfits
    cv=3
)
cal_clf.fit(X_train_c, y_train_c)

def get_prob(model, X):
    return model.predict_proba(X)[:, 1]

val_prob_base = get_prob(base_clf, X_val_c)
test_prob_base = get_prob(base_clf, X_test_c)

val_prob_cal = get_prob(cal_clf, X_val_c)
test_prob_cal = get_prob(cal_clf, X_test_c)

ap_val_base = average_precision_score(y_val_c, val_prob_base)
ap_test_base = average_precision_score(y_test_c, test_prob_base)

ap_val_cal = average_precision_score(y_val_c, val_prob_cal)
ap_test_cal = average_precision_score(y_test_c, test_prob_cal)

print("AP (PR-AUC) base: VAL", round(ap_val_base, 4), "TEST", round(ap_test_base, 4))
print("AP (PR-AUC) cal : VAL", round(ap_val_cal, 4), "TEST", round(ap_test_cal, 4))

## Step 8 — PR curve + Calibration

### PR curve (Precision–Recall)
For rare events, PR curve is the most informative:
- Moving along the curve corresponds to different thresholds.
- Higher recall usually means more false alarms (lower precision).

### Calibration curve
Calibration answers: “If the model says 30%, is it *actually* 30%?”
This matters for decision-making and interpretability.

In [ ]:
def plot_pr(y_true, probs, title):
    prec, rec, thr = precision_recall_curve(y_true, probs)
    ap = average_precision_score(y_true, probs)
    plt.figure(figsize=(7,4))
    plt.plot(rec, prec, label=f"AP={ap:.3f}")
    plt.title(title)
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.grid(True)
    plt.legend()
    plt.show()
    return prec, rec, thr, ap

print("Calibrated model PR curves:")
prec_v, rec_v, thr_v, ap_v = plot_pr(y_val_c, val_prob_cal, "PR Curve (Validation) — Calibrated")
prec_t, rec_t, thr_t, ap_t = plot_pr(y_test_c, test_prob_cal, "PR Curve (Test) — Calibrated")

In [ ]:
def plot_calibration(y_true, probs, title):
    frac_pos, mean_pred = calibration_curve(y_true, probs, n_bins=10, strategy="quantile")
    plt.figure(figsize=(6,4))
    plt.plot(mean_pred, frac_pos, marker="o")
    plt.plot([0,1],[0,1])
    plt.title(title)
    plt.xlabel("Mean predicted probability")
    plt.ylabel("Fraction of positives")
    plt.grid(True)
    plt.show()

print("Brier score (lower is better):", round(brier_score_loss(y_test_c, test_prob_cal), 4))
plot_calibration(y_test_c, test_prob_cal, "Calibration curve (TEST) — Calibrated")

## Step 9 — Threshold tuning (operational choice)

Default threshold 0.5 is rarely optimal for imbalanced events.

We choose thresholds based on:
- **Best F1** on validation (balanced screening rule)
- **High recall** (early warning mode)
- **Higher precision** (false-alarm control / triage mode)

We then report VAL + TEST metrics for each threshold and show the confusion outcomes.

In [ ]:
def metrics_at_threshold(y_true, probs, thr: float) -> Dict[str, float]:
    y_pred = (probs >= thr).astype(int)
    p = precision_score(y_true, y_pred, zero_division=0)
    r = recall_score(y_true, y_pred, zero_division=0)
    f = f1_score(y_true, y_pred, zero_division=0)
    cm = confusion_matrix(y_true, y_pred)  # [[TN, FP],[FN, TP]]
    tn, fp, fn, tp = cm.ravel()
    return {
        "precision": p, "recall": r, "f1": f,
        "TN": tn, "FP": fp, "FN": fn, "TP": tp,
        "alerts": int(tp + fp)
    }

# 1) Best-F1 threshold on validation
prec, rec, thr = precision_recall_curve(y_val_c, val_prob_cal)
# thr length = len(prec)-1
f1s = (2 * prec[:-1] * rec[:-1]) / (prec[:-1] + rec[:-1] + 1e-12)
best_idx = np.argmax(f1s)
thr_best_f1 = thr[best_idx]

# 2) High recall target
target_recall = 0.80
idx_recall = np.where(rec[:-1] >= target_recall)[0]
thr_high_recall = thr[idx_recall[-1]] if len(idx_recall) else None

# 3) Higher precision target
target_precision = 0.20
idx_prec = np.where(prec[:-1] >= target_precision)[0]
thr_high_prec = thr[idx_prec[0]] if len(idx_prec) else None

thresholds = {
    "Best-F1 (VAL)": float(thr_best_f1),
    "High-Recall (VAL)": float(thr_high_recall) if thr_high_recall is not None else np.nan,
    "High-Precision (VAL)": float(thr_high_prec) if thr_high_prec is not None else np.nan
}

rows = []
for name, tval in thresholds.items():
    if not np.isfinite(tval):
        continue
    val_m = metrics_at_threshold(y_val_c, val_prob_cal, tval)
    test_m = metrics_at_threshold(y_test_c, test_prob_cal, tval)
    rows.append({
        "choice": name,
        "threshold": tval,
        "VAL_precision": val_m["precision"],
        "VAL_recall": val_m["recall"],
        "VAL_f1": val_m["f1"],
        "TEST_precision": test_m["precision"],
        "TEST_recall": test_m["recall"],
        "TEST_f1": test_m["f1"],
        "TEST_FP": test_m["FP"],
        "TEST_TP": test_m["TP"],
        "TEST_alerts": test_m["alerts"],
    })

comparison = pd.DataFrame(rows).sort_values("TEST_f1", ascending=False)
comparison.style.format({
    "threshold": "{:.6f}",
    "VAL_precision": "{:.3f}",
    "VAL_recall": "{:.3f}",
    "VAL_f1": "{:.3f}",
    "TEST_precision": "{:.3f}",
    "TEST_recall": "{:.3f}",
    "TEST_f1": "{:.3f}",
})

## Step 10 — Operating threshold (locked)

We choose **one default threshold** to make the project operationally complete.

Recommendation:
- **Best-F1 threshold** as the default “balanced screening rule”

Operational alternatives:
- Use **High-Recall** for early warning (accept high alert volume)
- Use **High-Precision** for triage efficiency (fewer alerts, more misses)

In [ ]:
LOCKED_THRESHOLD = thresholds["Best-F1 (VAL)"]
LOCKED_THRESHOLD

## Step 11 — Exposure mapping (simple gridding)

To visualize smoke exposure geographically, we convert station PM2.5 into a grid.

This is not a full atmospheric dispersion model. It’s a **data product**:
- take station PM2.5 for a chosen day
- interpolate to a lat/lon grid using inverse-distance weighting (IDW)

This delivers a clear “NASA-style” map output for a portfolio.

If you later add smoke polygons / satellite AOD, you can make this stronger.

In [ ]:
def idw_interpolate(xy: np.ndarray, values: np.ndarray, grid_xy: np.ndarray, power: float = 2.0, eps: float = 1e-12):
    # xy shape (n,2), grid_xy shape (m,2)
    out = np.zeros(grid_xy.shape[0], dtype=float)
    for i, g in enumerate(grid_xy):
        d = np.sqrt(((xy - g) ** 2).sum(axis=1)) + eps
        w = 1.0 / (d ** power)
        out[i] = np.sum(w * values) / np.sum(w)
    return out

# Pick a day from TEST for map demo
map_day = test_df["date"].sort_values().iloc[-30]  # ~one month before end
day_df = daily.copy()
day_df["date"] = pd.to_datetime(day_df["date"], utc=True)
day_slice = day_df[day_df["date"] == map_day].dropna(subset=["pm25_mean"]).copy()

print("Map day:", map_day)
print("Stations on day:", len(day_slice))

# Grid bounds
lat_min, lat_max = day_slice["lat"].min(), day_slice["lat"].max()
lon_min, lon_max = day_slice["lon"].min(), day_slice["lon"].max()

# Coarse grid for speed (increase resolution later)
n_lat, n_lon = 60, 80
grid_lats = np.linspace(lat_min, lat_max, n_lat)
grid_lons = np.linspace(lon_min, lon_max, n_lon)
grid_xy = np.array([(la, lo) for la in grid_lats for lo in grid_lons])

xy = day_slice[["lat","lon"]].values
vals = day_slice["pm25_mean"].values

grid_vals = idw_interpolate(xy, vals, grid_xy, power=2.0)
grid_vals = grid_vals.reshape(n_lat, n_lon)

plt.figure(figsize=(9,4))
plt.imshow(
    grid_vals,
    origin="lower",
    extent=[lon_min, lon_max, lat_min, lat_max],
    aspect="auto"
)
plt.scatter(day_slice["lon"], day_slice["lat"], s=10, alpha=0.7)
plt.title(f"Estimated PM2.5 exposure surface (IDW) — {map_day.date()}")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.colorbar(label="PM2.5 (µg/m³)")
plt.grid(False)
plt.show()

## Results Summary (Final)

### What worked
- Time-safe lag features capture persistence and rapid changes.
- PR-AUC provides the correct view for rare exceedance prediction.
- Threshold tuning turns a model score into an operational decision rule.
- Exposure mapping produces a portfolio-ready visual product.

### What to report in GitHub / LinkedIn
- Best regression model and MAE/RMSE on TEST
- Classification PR-AUC and threshold table
- The locked operating threshold + the two operational alternatives
- A map example day showing exposure surface

### Limitations
- Station coverage is uneven; interpolation is not a physical dispersion model.
- Smoke signal is stronger when you add NOAA HMS smoke polygons / AOD.
- Meteorology improves generalization and should be added for production.

### Next steps (future iteration, optional)
- Add NOAA HMS smoke polygons → station smoke density feature
- Add AOD (satellite) or wind trajectory features
- Cross-validation grouped by geography + time
- Deploy a simple CLI/predict script for next-day alerts